In [ ]:
import copy
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = load_day(1)

In [ ]:
def fuel(mass):
    return mass // 3 - 2

__Part 1__

In [ ]:
total = 0

for elem in map(int, content):
    total += fuel(elem)
    
print(f'Answer 1: {total}')

__Part 2__

In [ ]:
total = 0

for elem in map(int, content):
    fuel_mass = fuel(elem)
    while fuel_mass > 0:
        total += fuel_mass
        fuel_mass = fuel(fuel_mass)

print(f'Answer 2: {total}')

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = load_day(2)
content = list(map(int, content[0].split(',')))
N = len(content)

__Part 1__

In [ ]:
def program(noun=12, verb=2):
    content[1] = noun
    content[2] = verb

    solution = copy.deepcopy(content)

    for idx in range(0, N, 4):
        if content[idx] == 99:
            break
        elif solution[idx] == 1:
            solution[solution[idx+3]] = solution[solution[idx+1]] + solution[solution[idx+2]]
        elif solution[idx] == 2:
            solution[solution[idx+3]] = solution[solution[idx+1]] * solution[solution[idx+2]]
        else:
            print("error, code:", solution[idx], " position: ", idx)
            
    return solution

In [ ]:
print('Anwser 1: {}'.format(program()[0]))

__Part 2__

In [ ]:
for noun in range(100):
    for verb in range(100):
        run = program(noun, verb)
        if run[0] == 19690720:
            print('Answer 2: {}'.format(run[1] * 100 + run[2]))

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)
ex1 = ["R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83"]
ex2 = ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"]

In [ ]:
content = [ c.split(',') for c in content]
ex1 = [ c.split(',') for c in ex1]
ex2 = [ c.split(',') for c in ex2]

In [ ]:
def order_to_coords(value, from_coord):
    orient = value[0]
    length = int(value[1:])
    
    if orient == 'R':
        return [(from_coord[0] + x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'L':
        return [(from_coord[0] - x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'U':
        return [(from_coord[0], from_coord[1] + x) for x in range(1, length + 1)]
    elif orient == 'D':
        return [(from_coord[0], from_coord[1] - x) for x in range(1, length + 1)]

__Part 1__

In [ ]:
paths = []

for row in content:
    cur_pos = (0, 0)
    path = []

    for order in row:
        points = order_to_coords(order, cur_pos)
        path += points
        
        cur_pos = points[-1]
        
    paths += [path]

In [ ]:
intersections = set(paths[0]).intersection(set(paths[1]))
sorted_intersections = sorted(intersections, key=lambda x: abs(x[0]) + abs(x[1]))
print('Answer 1: {}'.format(abs(sorted_intersections[0][0]) + abs(sorted_intersections[0][1])))

__Part 2__

In [ ]:
distances = []
for inter in intersections:
    distances += [2 + paths[0].index(inter) + paths[1].index(inter)] # initial step for each path is not considered
    
print('Answer 2: {}'.format(min(distances)))

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
# content = load_day(4)
puzzle_input = "145852-616942"

In [ ]:
list("test")

__Part 1__

In [ ]:
def check_input(value):
    if not isinstance(value, str):
        raise ValueError("input should be string")
        
    elems = list(value)
    
    prev_elem = None
    before_elem = None
    
    adjacent = False
    decrease = False
    
    for e in elems:
        if prev_elem is not None:
            if not adjacent:
                adjacent = prev_elem == e
            if not decrease:
                decrease = ord(prev_elem) > ord(e)

        prev_elem = e
    return adjacent and not decrease

In [ ]:
candidates = []
for v in range(*map(int, puzzle_input.split('-'))):
    inp = str(v)
    candidates += [inp] if check_input(inp) else []
print(f'Answer 1: {len(candidates)}')

__Part 2__

In [ ]:
def lengths(value):
    le = []
    char = ''
    count = 1
    for v in value[1:]:
        if v == char:
            count += 1
        else:
            le.append(count)
            count = 1
            
        char = v
    return 

In [ ]:
leng

In [ ]:
for c in candidates:
    
    total += int(check_input2(str(v)))
print(f'Answer 2: {total}')

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
content = load_day(5)

__Part 1__

__Part 2__

## Day 6

http://adventofcode.com/2018/day/6

In [ ]:
content = load_day(6)

__Part 1__

__Part 2__

## Day 7

http://adventofcode.com/2018/day/7

In [ ]:
content = load_day(7)

__Part 1__

__Part 2__

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)

__Part 1__

__Part 2__